<a href="https://colab.research.google.com/github/FranMoretti/FranMoretti/blob/main/03_creaci%C3%B3n_de_perfiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recomendación por contenidos - Creación de perfiles

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
conn = sqlite3.connect('data.db')

df_genero = pd.read_sql_query(f"""
SELECT genero, AVG(rating) / 10.0 AS rating
  FROM interacciones AS i JOIN libros AS l ON i.id_libro = l.id_libro
  WHERE id_lector = '{id_lector}'
GROUP BY 1
HAVING count(*) >= 3
ORDER BY count(*) DESC
LIMIT 10
""", conn)
df_genero["rating"] = df_genero["rating"] / df_genero["rating"].values.sum()
df_genero.head(10)
#acá lo estoy normalizando
# y la idea también es hacerlo cuando hago la query por autor para despues sumar ambos o algún tipo de operación


,genero,rating
0,"Novela negra, intriga, terror",0.097495
1,Narrativa,0.099337
2,Histórica y aventuras,0.100779
3,Literatura contemporánea,0.102417
4,"Fantástica, ciencia ficción",0.103046
5,Ficción literaria,0.094352
6,Infantil y juvenil,0.103100
7,Clásicos de la literatura,0.104773
8,"Romántica, erótica",0.093218
9,Ensayo,0.101482


In [ ]:
dict_generos = dict(zip(df_genero["genero"].values.tolist(), df_genero["rating"].values.tolist()))
dict_generos

{'Novela negra, intriga, terror': 0.09749545438207294,
 'Narrativa': 0.0993367730101395,
 'Histórica y aventuras': 0.1007793545495927,
 'Literatura contemporánea': 0.10241703679184956,
 'Fantástica, ciencia ficción': 0.10304594093546689,
 'Ficción literaria': 0.09435178481161946,
 'Infantil y juvenil': 0.10310010732632985,
 'Clásicos de la literatura': 0.1047733413545366,
 'Romántica, erótica': 0.09321794391937721,
 'Ensayo': 0.10148226291901521}

In [ ]:
df_autor = pd.read_sql_query(f"""
SELECT autor, AVG(rating) / 10.0 AS rating
  FROM interacciones AS i JOIN libros AS l ON i.id_libro = l.id_libro
  WHERE autor IS NOT NULL AND id_lector = '{id_lector}'
GROUP BY 1
  HAVING count(*) >= 3
  ORDER BY count(*) DESC
LIMIT 10
""", conn)
df_autor["rating"] = df_autor["rating"] / df_autor["rating"].values.sum()
df_autor.head(10)

,autor,rating
0,"KING, STEPHEN",0.099772
1,"ROWLING, JOANNE K.",0.106186
2,"FOLLETT, KEN",0.102845
3,"PÉREZ-REVERTE, ARTURO",0.094708
4,"GARCÍA MÁRQUEZ, GABRIEL",0.103909
5,"RUIZ ZAFÓN, CARLOS",0.101586
6,"TOLKIEN, J.R.R. (John Ronald Reuel Tolkien)",0.108635
7,"ALLENDE, ISABEL",0.098046
8,"BROWN, DAN",0.083093
9,"CHRISTIE, AGATHA",0.101219


In [ ]:
dict_autores = dict(zip(df_autor["autor"].values.tolist(), df_autor["rating"].values.tolist()))
dict_autores

{'KING, STEPHEN': 0.09977203354244651,
 'ROWLING, JOANNE K.': 0.10618633616007588,
 'FOLLETT, KEN': 0.10284500628031046,
 'PÉREZ-REVERTE, ARTURO': 0.09470837021249644,
 'GARCÍA MÁRQUEZ, GABRIEL': 0.10390875714764995,
 'RUIZ ZAFÓN, CARLOS': 0.10158602182122309,
 'TOLKIEN, J.R.R. (John Ronald Reuel Tolkien)': 0.10863490946718009,
 'ALLENDE, ISABEL': 0.09804593097832037,
 'BROWN, DAN': 0.08309344524619577,
 'CHRISTIE, AGATHA': 0.10121918914410137}

In [ ]:
id_lector = "c3po"

generos = "'" + "', '".join(dict_generos.keys()) + "'"
autores = "'" + "', '".join(dict_autores.keys()) + "'"

df = pd.read_sql_query(f"""
SELECT id_libro, genero, autor
  FROM libros
 WHERE id_libro NOT IN (SELECT id_libro FROM interacciones WHERE id_lector = '{id_lector}')
   AND (genero IN ({generos}) OR autor IN ({autores}))
""", conn)
df.head()

,id_libro,genero,autor
0,las-particulas-elementales,Narrativa,"HOUELLEBECQ, MICHEL"
1,quien-domina-el-mundo,Ensayo,"CHOMSKY, NOAM"
2,antimanual-de-filosofia,Ensayo,"ONFRAY, MICHEL"
3,momentos-estelares-de-la-humanidad-catorce-min...,Histórica y aventuras,"ZWEIG, STEFAN"
4,serotonina,Narrativa,"HOUELLEBECQ, MICHEL"


In [ ]:
df['genero_rating'] = df['genero'].map(lambda x: dict_generos.get(x, 0))
df['autor_rating'] = df['autor'].map(lambda x: dict_autores.get(x, 0))
df['rating'] = df['genero_rating'] + df['autor_rating']
df.sort_values(by='rating', ascending=False, inplace=True)
df.head()
#acá esta recomendando con efecto túnel, solo en base a lo que leyó el id_lector, en caso que quiera
#recomendarle autores similares debería hacer una matriz de autor contra autor (o sea de a pares)
#donde vea cuantas veces aparecen repetidos

,id_libro,genero,autor,genero_rating,autor_rating,rating
29248,roverandom-y-el-senor-bliss,Infantil y juvenil,"TOLKIEN, J.R.R. (John Ronald Reuel Tolkien)",0.103100,0.108635,0.211735
26031,cartas-de-papa-noel,Infantil y juvenil,"TOLKIEN, J.R.R. (John Ronald Reuel Tolkien)",0.103100,0.108635,0.211735
29257,el-senor-bliss,Infantil y juvenil,"TOLKIEN, J.R.R. (John Ronald Reuel Tolkien)",0.103100,0.108635,0.211735
1666,los-hijos-de-hurin,"Fantástica, ciencia ficción","TOLKIEN, J.R.R. (John Ronald Reuel Tolkien)",0.103046,0.108635,0.211681
6407,el-hobbit-edicion-especial-75-aniversario,"Fantástica, ciencia ficción","TOLKIEN, J.R.R. (John Ronald Reuel Tolkien)",0.103046,0.108635,0.211681
